# Using an Echo-State Network to predict ENSO

In [1]:
cd("$(homedir())/Documents/enso_project.jl")
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\lihel\Documents\enso_project.jl`


In [2]:
using ReservoirComputing, CSV, DataFrames, DynamicalSystems, Plots, enso_project

┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   Base.PkgId(Base.UUID("e3ecd195-ca82-5397-9546-f380c1e34951"), "NonlinearSolveBaseSparseMatrixColoringsExt")
│   Base.PkgId(Base.UUID("385e4588-a1a0-5c1d-98fa-d45bf6f8ecf9"), "LinearSolveKernelAbstractionsExt")
│   Base.PkgId(Base.UUID("3bcf3b12-2128-5d18-8b3b-bcdd6f83637b"), "WeightInitializersGPUArraysExt")
│   Base.PkgId(Base.UUID("b00db79b-61e3-50fb-b26f-2d35b2d9e4ed"), "DiffEqBaseChainRulesCoreExt")
│   Base.PkgId(Base.UUID("8913a72c-1f9b-4ce2-8d82-65094dcecaec"), "NonlinearSolve")
│   Base.PkgId(Base.UUID("7edab7de-1038-5e4f-97a7-6bfc75d44324"), "NonlinearSolveQuasiNewtonForwardDiffExt")
│   Base.PkgId(Base.UUID("1dea7af3-3e70-54e6-95c3-0bf5283fa5ed"), "OrdinaryDiffEq")
│   Base.PkgId(Base.UUID("0d7ed370-da01-4f52-bd93-41d350b8b718"), "StaticArrayInterface")
│   Base.PkgId(Base.UUID("693f0f32-89f9-59b4-b981-3d79b82ef24b"), "SparseDiffToolsEnzymeExt")
│   Base.PkgId(Base.UUID("63d416d0-6995-5965-81e0-5

In [ ]:
# read whole input data frame
df_data = CSV.read("data/sst_34_anomaly_embedded.txt", DataFrame; delim=',', ignorerepeated=true)

# bring into correct format
data = Matrix(transpose(Matrix(df_data)))

## Network Training
We train the network in the same manner for six different data splits. For each data split, we try our a set of different hyperparameters.

### Hyperparameter Tuning
For each data split, we choose suitable hyperparameters by performing a grid search.


We create the hyperparameter grid as follows:
- We observe that too big reservoir sizes cause singular matrices in the linear regression (due to too little training data), thus we adapt the reservoir size to be adequately small compared to the amount of training data. When performing the regression we note that smaller reservoir sizes are favoured in our use case.
- to ensure the Echo State Property the spectral radius should be smaller than 1 (unless long memory is required). We try out different combinations.
- a sparsity of 0.1 is usually recommended, we test different values around 0.1
- a input scale of 0.1 is recommended by the literature
- we try ridge params as suggested in the lecture

In [3]:
# set up parameter grid
reservoir_sizes = [60, 70, 80, 90, 95, 100, 105, 110, 115, 120, 130]
spectral_radii = [0.8, 0.9, 1.0]
sparsities = [0.05, 0.08, 0.1, 0.12, 0.14]
input_scales = [0.1]
ridge_values = [0.0, 1e-6, 1e-5]

param_grid = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)

495-element Vector{Any}:
 enso_project.ESNHyperparams(60, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(70, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(80, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(90, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(95, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(100, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(105, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(110, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(115, 0.8, 0.05, 0.1, 0.0)
 enso_project.ESNHyperparams(120, 0.8, 0.05, 0.1, 0.0)
 ⋮
 enso_project.ESNHyperparams(80, 1.0, 0.14, 0.1, 1.0e-5)
 enso_project.ESNHyperparams(90, 1.0, 0.14, 0.1, 1.0e-5)
 enso_project.ESNHyperparams(95, 1.0, 0.14, 0.1, 1.0e-5)
 enso_project.ESNHyperparams(100, 1.0, 0.14, 0.1, 1.0e-5)
 enso_project.ESNHyperparams(105, 1.0, 0.14, 0.1, 1.0e-5)
 enso_project.ESNHyperparams(110, 1.0, 0.14, 0.1, 1.0e-5)
 enso_project.ESNHyperparams(115, 1.0, 0.14, 0.1, 1.0e-5)
 enso_project.ESNHyperpa

### Training Data Split 20%

train = 20%,
val =  70%,
test = 10%

In [6]:
# read input data
df_train_data_20 = CSV.read("data/sst_34_data_split_20/train_sst_34_anomaly_embedded_20.txt", DataFrame; delim=',', ignorerepeated=true)
df_val_data_20 = CSV.read("data/sst_34_data_split_20/val_sst_34_anomaly_embedded_20.txt", DataFrame; delim=',', ignorerepeated=true)
df_test_data_20 = CSV.read("data/sst_34_data_split_20/test_sst_34_anomaly_embedded_20.txt", DataFrame; delim=',', ignorerepeated=true)

# bring into correct format
train_data_20 = Matrix(transpose(Matrix(df_train_data_20)))
val_data_20 = Matrix(transpose(Matrix(df_val_data_20)))
test_data_20 = Matrix(transpose(Matrix(df_test_data_20)))

5×49 Matrix{Float64}:
  0.67   0.71   0.97   0.72   0.75  …  -0.93  -0.85  -0.93  -0.84  -0.69
  0.42   0.15  -0.01   0.52   0.46     -0.01   0.19   0.47   0.88   1.07
  0.52   0.35   0.46   0.41  -0.21      1.53   1.59   1.9    1.99   1.78
 -0.14  -0.52  -0.64  -1.13  -1.23      1.24   0.81   0.31   0.24   0.21
 -0.9   -0.87  -0.55  -0.57  -0.32     -0.15  -0.28  -0.14  -0.62  -0.71

In [8]:
# network training
esn_20, W_out_20 = enso_project.cross_validate_esn(train_data_20, val_data_20, param_grid)

enso_project.ESNHyperparams(60, 0.8, 0.05, 0.1, 0.0)
Validation loss = 7.201424571500899e8
enso_project.ESNHyperparams(70, 0.8, 0.05, 0.1, 0.0)
Validation loss = 1.2341563365429726e8


LinearAlgebra.SingularException: LinearAlgebra.SingularException(99)

In [ ]:
enso_project.plot_esn_prediction(esn_20, W_out_20, test_data_20)

We now also plot the ESN against the validation data set.

In [ ]:
enso_project.plot_esn_prediction(esn, W_out, val_data)